In [1]:
import os, random
import ssl
import urllib.request

ssl._create_default_https_context = ssl._create_unverified_context


if not os.path.exists('input.txt'):
    names_url='https://raw.githubusercontent.com/karpathy/makemore/refs/heads/master/names.txt'
    urllib.request.urlretrieve(names_url, "input.txt")
docs = [l.strip() for l in open('input.txt').read().strip().split('\n') if l.strip()]
random.shuffle(docs)
print(f"num docs:{len(docs)}")


num docs:32033


In [2]:
import math

In [3]:
#tokeniser
uchars=sorted(set(''.join(docs)))
BOS= len(uchars)
vocab_size= BOS + 1 

In [4]:
class Value:
    __slots__ = ('data','grad','_children','_local_grads')

    def __init__(self, data,children=(),local_grads=()):
        self.data=data
        self.grad=0
        self._children = children
        self._local_grads= local_grads
    
    def __add__(self,other): #x+3 x.__add__(3), x=Value(4), self.__add__(other)
        # print("calling,__add__")
        other=other if isinstance(other,Value) else Value(other) # 3becomes Value(3)
        return Value(self.data + other.data, (self,other), (1,1))
    
    def __radd__(self,others): #swaps 3+x to x+3
        return self+others
    
    def __mul__(self,other):
        # print("calling __multiply__")
        other=other if isinstance(other,Value) else Value(other)
        return Value(self.data * other.data, (self,other),(other.data,self.data)) #deravitive of x=y.data, y=x.data
    
    def __rmul__(self, other): return self * other

    def __pow__(self, other):
        return Value(
            self.data ** other,
            (self,),
            (other * (self.data ** (other - 1)),)
        )



    def log(self): return Value(math.log(self.data),(self,),(1/self.data,))

    def exp(self): return Value(math.exp(self.data), (self,), (math.exp(self.data),))
    
    def __neg__(self):
        return self *-1
    
    def __sub__(self,other): return self +(-other)

    def __truediv__(self,other): return self * other**-1

    def __rtruediv__(self,other): return other * self**-1

    def relu(self):
        return Value(max(0,self.data),(self,),(float(self.data >0),))
    
    '''
    x ----\
        + ---- a ----\
y ----/               *
                       \
                        z
w ---------------------/
z.grad=1

    '''
    
    def backward(self):
        topo=[]
        visited=set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._children:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad=1
        for v in reversed(topo):
            for child, local_grad in zip(v._children, v._local_grads):
                child.grad+=local_grad*v.grad







In [5]:
a = Value(2.0)
b = Value(3.0)
c = a * b       # c = 6.0
L = c + a       # L = 8.0
L.backward()
print(a.grad)   # 4.0 (dL/da = b + 1 = 3 + 1, via both paths). L=ab+a
print(b.grad)   # 2.0 (dL/db = a = 2)

4.0
2.0


In [6]:
#Parameters
n_embed=16
n_head=4
n_layer=1
block_size=16
head_dim=n_embed//n_head
'''shape = (B, 16, 16)
         ↑   ↑   ↑
         |   |   |
         |   |   embedding dimension
         |   sequence length(block_size)
         batch
'''
matrix=lambda nout,nin,std=0.08:[
    [Value(random.gauss(0,std)) for _ in range(nin)] 
    for _ in range (nout)
    ]
state_dict ={'wte':matrix(vocab_size,n_embed), 'wpe': matrix(block_size,n_embed),'lm_head': matrix(vocab_size,n_embed)}
#wte = Word Token Embedding matrix
#wpe = Word Position Embedding matrix

for i in range(n_layer):
    state_dict[f'layer{i}.attn_wq'] = matrix(n_embed,n_embed)
    state_dict[f'layer{i}.attn_wk'] = matrix(n_embed,n_embed)
    state_dict[f'layer{i}.attn_wv'] = matrix(n_embed,n_embed)
    state_dict[f'layer{i}.attn_wo'] = matrix(n_embed,n_embed)
    '''Input embeddings → produce Q, K, V

Compute attention per head

Concatenate all head outputs

Apply a final linear projection'''
    state_dict[f'layer{i}.mlp_fc1'] = matrix(4*n_embed,n_embed)
    state_dict[f'layer{i}.mlp_fc2'] = matrix(n_embed,4*n_embed)

parms = [p for mat in state_dict.values() for row in mat for p in row]

len(parms)




4192

In [7]:
def linear(x,w): #transformation
    return [sum(wi * xi for wi,xi in zip(wo,x)) for wo in w]

In [8]:
def softmax(logits):
    max_val= max(val.data for val in logits)
    exp= [(val-max_val).exp() for val in logits]
    total= sum(exp)
    return [e/total for e in exp]

In [9]:
def rmsnorm(x):
    ms=sum(xi*xi for xi in x)/len(x)
    scale=(ms+1e-5)**-0.5
    return [xi * scale for xi in x]



This is because modern transformers use **Pre-Norm architecture**.

In a transformer layer, the structure looks like this:

So per layer:

* x→ Norm → Attention → Add residual

Then:
* Norm → MLP → Add residual

So each layer has two normalizations.


In [10]:
def gpt(token_id, pos_id, keys, values):
    tok_emb = state_dict['wte'][token_id] # token embedding
    pos_emb = state_dict['wpe'][pos_id] # position embedding
    x = [t + p for t, p in zip(tok_emb, pos_emb)] # joint token and position embedding
    x = rmsnorm(x)

    for li in range(n_layer):
        # 1) Multi-head attention block
        x_residual = x
        x = rmsnorm(x)
        q = linear(x, state_dict[f'layer{li}.attn_wq'])
        k = linear(x, state_dict[f'layer{li}.attn_wk'])
        v = linear(x, state_dict[f'layer{li}.attn_wv'])
        keys[li].append(k)
        values[li].append(v)
        x_attn = []
        for h in range(n_head):
            hs = h * head_dim
            q_h = q[hs:hs+head_dim]
            k_h = [ki[hs:hs+head_dim] for ki in keys[li]]
            v_h = [vi[hs:hs+head_dim] for vi in values[li]]
            attn_logits = [sum(q_h[j] * k_h[t][j] for j in range(head_dim)) / head_dim**0.5 for t in range(len(k_h))]
            attn_weights = softmax(attn_logits)
            head_out = [sum(attn_weights[t] * v_h[t][j] for t in range(len(v_h))) for j in range(head_dim)]
            x_attn.extend(head_out)
        x = linear(x_attn, state_dict[f'layer{li}.attn_wo'])
        x = [a + b for a, b in zip(x, x_residual)]
        # 2) MLP block
        x_residual = x
        x = rmsnorm(x)
        x = linear(x, state_dict[f'layer{li}.mlp_fc1'])
        x = [xi.relu() for xi in x]
        x = linear(x, state_dict[f'layer{li}.mlp_fc2'])
        x = [a + b for a, b in zip(x, x_residual)]

    logits = linear(x, state_dict['lm_head'])
    return logits

In [11]:
learning_rate,beta1,beta2,eps_adam=0.01,0.85,0.99,1e-8
m=[0.0] * len(parms)
v=[0.0] * len(parms)

nums_steps=1000
for step in range(nums_steps):
    doc=docs[step%len(docs)]
    tokens=[BOS]+[uchars.index(ch) for ch in doc] +[BOS]
    n= min(block_size,len(tokens)-1)
    keys, values = [[]for _ in range(n_layer)],[[] for _ in range(n_layer)]
    losses=[]
    for pos_id in range(n):
        token_id, target_id = tokens[pos_id], tokens[pos_id +1]
        logits=gpt(token_id,pos_id,keys,values)
        probs= softmax(logits)
        loss_t= -probs[target_id].log()
        losses.append(loss_t)
    loss=(1/n)*sum(losses)

    loss.backward()

    lr_t=learning_rate*(1-step/nums_steps) # lr decay

    for i, p in enumerate(parms):
        m[i] = beta1 * m[i] + (1 - beta1) * p.grad
        v[i] = beta2 * v[i] + (1 - beta2) * p.grad ** 2
        m_hat = m[i] / (1 - beta1 ** (step + 1))
        v_hat = v[i] / (1 - beta2 ** (step + 1))
        p.data -= lr_t * m_hat / (v_hat ** 0.5 + eps_adam)
        p.grad = 0



    print(f"step {step+1:4d} / {nums_steps:4d} | loss {loss.data:.4f}")
    

step    1 / 1000 | loss 3.4826
step    2 / 1000 | loss 3.4193
step    3 / 1000 | loss 3.2604
step    4 / 1000 | loss 2.9972
step    5 / 1000 | loss 3.3197
step    6 / 1000 | loss 3.3560
step    7 / 1000 | loss 3.2189
step    8 / 1000 | loss 3.1300
step    9 / 1000 | loss 3.1470
step   10 / 1000 | loss 3.0812
step   11 / 1000 | loss 3.1643
step   12 / 1000 | loss 3.0338
step   13 / 1000 | loss 3.2901
step   14 / 1000 | loss 3.2390
step   15 / 1000 | loss 2.9296
step   16 / 1000 | loss 3.3146
step   17 / 1000 | loss 3.1667
step   18 / 1000 | loss 2.2688
step   19 / 1000 | loss 3.0348
step   20 / 1000 | loss 2.8102
step   21 / 1000 | loss 3.3720
step   22 / 1000 | loss 2.4961
step   23 / 1000 | loss 2.9241
step   24 / 1000 | loss 2.5861
step   25 / 1000 | loss 2.3197
step   26 / 1000 | loss 2.8641
step   27 / 1000 | loss 3.1677
step   28 / 1000 | loss 1.9265
step   29 / 1000 | loss 2.8918
step   30 / 1000 | loss 3.0055
step   31 / 1000 | loss 2.5147
step   32 / 1000 | loss 2.8298
step   3

In [14]:
temperature=0.5
print("\n--- inference (new, hallucinated names) ---")
for sample_idx in range(20):
    keys, values = [[]for _ in range(n_layer)],[[] for _ in range(n_layer)]
    token_id=BOS
    sample=[]
    for pos_id in range(block_size):
        logits= gpt(token_id,pos_id,keys,values)
        prbs=softmax([l/temperature for l in logits])
        token_id= random.choices(range(vocab_size),weights=[p.data for p in prbs])[0]
        # token_id = max(
        #     range(vocab_size),
        #     key=lambda i: probs[i].data
        # )
        if token_id==BOS:
            break
        sample.append(uchars[token_id])
    
    print(f"sample {sample_idx+1:2d}: {''.join(sample)}")


--- inference (new, hallucinated names) ---
sample  1: kindah
sample  2: aemis
sample  3: kaaia
sample  4: jalil
sample  5: kelin
sample  6: jasha
sample  7: li
sample  8: areeli
sample  9: kerona
sample 10: cori
sample 11: jadan
sample 12: naele
sample 13: reali
sample 14: leela
sample 15: aveli
sample 16: jaril
sample 17: aranon
sample 18: shalya
sample 19: aacth
sample 20: aronai
